In [1]:
%matplotlib inline
from python.analysis import Master, vector, Plots
import apps.prod4a_merge_study as merge_study

import awkward as ak
import numpy as np

from rich import print


def GetSharedHitsMask(particleData : Master.ParticleData, e, a, b):
    #! returns the mask for particle b only
    total_mask = None
    for c1, t1 in zip(particleData.channel[e][a], particleData.peakTime[e][a]):
        mask = (c1 == particleData.channel[e][b]) & (t1 == particleData.peakTime[e][b])
        if total_mask is None:
            total_mask = mask
        else:
            total_mask = total_mask | mask
    return total_mask


def GetSharedEnergy(particleData : Master.ParticleData, e, a, b):
    mask_a = GetSharedHitsMask(particleData, e, b, a)
    mask_b = GetSharedHitsMask(particleData, e, a, b)

    energy_a = particleData.hit_energy[e][a][mask_a]
    energy_b = particleData.hit_energy[e][b][mask_b]

    sum_a = ak.sum(energy_a[energy_a > 0])
    sum_b = ak.sum(energy_b[energy_b > 0])
    return sum_a, sum_b

def SharedHitsMetrics(particleData : Master.ParticleData, e, i, mask):
    print(f"number of hits shared with other shower: {ak.count(mask[mask])}")
    shared_hits_energy = particleData.hit_energy[e][i][mask]
    print(f"number of hits shared with valid energy: {ak.count(shared_hits_energy[shared_hits_energy > 0])}")

    hits_channel = particleData.channel[e][i][mask]
    hits_peakTime = particleData.peakTime[e][i][mask]

    shared_hits_energy = particleData.hit_energy[e][i][mask]
    shared_hits_integral = particleData.integral[e][i][mask]
    
    # print(f"{ak.sort(shared_hits_integral)=}")
    # print(f"{ak.sort(shared_hits_energy)=}")
    
    space_points = particleData.spacePoints[e][i][mask]
    
    # print(f"{ak.sort(space_points.x)=}")
    # print(f"{ak.sort(space_points.y)=}")
    # print(f"{ak.sort(space_points.z)=}")
    
    total_energy = ak.sum(shared_hits_energy[shared_hits_energy > 0])
    total_integral = ak.sum(shared_hits_integral[shared_hits_integral > 0])
    
    # print(f"shared hits total energy: {total_energy}")
    # print(f"shared hits total integral: {total_integral}")

    return {
        "hits_channel"          : hits_channel,
        "hits_peakTime"         : hits_peakTime,
        "shared_hits_energy"    : shared_hits_energy,
        "shared_hits_integral"  : shared_hits_integral,
        "space_points"          : space_points,
        "total_energy"          : total_energy,
        "total_integral"        : total_integral
        }

In [3]:
# events = Master.Data("work/ROOTFiles/Prod4a_1GeV_BeamSim_00_Selection.root", nEvents = 10)
events = Master.Data("work/ROOTFiles/debug/Prod4a_1GeV_BeamSim_00_Selection.root", nEvents = 10, start = 0)
events.io.ListNTuples()
# LoadAdditionalNTuplesNew(events, truth = True, reco = False)

merge_study.EventSelection(events)
start_showers, to_merge = merge_study.SplitSample(events)
signal, background, signal_all = merge_study.SignalBackground(events, start_showers, to_merge)

[
    'reco_PFP_ID',
    'reco_PFP_Mother',
    'reco_daughter_allShower_sliceID',
    'reco_daughter_allShower_beamCosmicScore',
    'reco_daughter_allSHower_PandoraPDG',
    'pandoraTag',
    'reco_daughter_PFP_emScore_collection',
    'reco_daughter_PFP_trackScore_collection',
    'CNNScore_collection',
    'reco_daughter_allShower_startX',
    'reco_daughter_allShower_startY',
    'reco_daughter_allShower_startZ',
    'reco_daughter_allShower_dirX',
    'reco_daughter_allShower_dirY',
    'reco_daughter_allShower_dirZ',
    'reco_daughter_allShower_coneAngle',
    'reco_daughter_allShower_length',
    'reco_daughter_PFP_nHits_collection',
    'reco_daughter_allShower_energy',
    'reco_daughter_allShower_calibrated_energy',
    'reco_daughter_allShower_hit_channel',
    'reco_daughter_allShower_hit_peakTime',
    'reco_daughter_allSHower_hit_integral',
    'reco_daughter_allShower_hit_energy',
    'reco_daughter_allShower_spacePointX',
    'reco_daughter_allShower_spacePointY',
    'reco_daughter_allShower_spacePointZ',
    'beamNum',
    'reco_beam_sliceID',
    'reco_beam_startX',
    'reco_beam_startY',
    'reco_beam_startZ',
    'reco_beam_endX',
    'reco_beam_endY',
    'reco_beam_endZ',
    'reco_beam_PFP_true_byHits_pdg',
    'reco_beam_PFP_true_byHits_startE',
    'reco_beam_PFP_true_byHits_mass',
    'reco_beam_PFP_true_byHits_startX',
    'reco_beam_PFP_true_byHits_startY',
    'reco_beam_PFP_true_byHits_startZ',
    'reco_beam_PFP_true_byHits_endX',
    'reco_beam_PFP_true_byHits_endY',
    'reco_beam_PFP_true_byHits_endZ',
    'reco_daughter_PFP_true_byHits_ID',
    'reco_daughter_PFP_true_byHits_Mother',
    'reco_daughter_PFP_true_byHits_pdg',
    'reco_daughter_PFP_true_byHits_Mother_pdg',
    'reco_daughter_PFP_true_byHits_startX',
    'reco_daughter_PFP_true_byHits_startY',
    'reco_daughter_PFP_true_byHits_startZ',
    'reco_daughter_PFP_true_byHits_endX',
    'reco_daughter_PFP_true_byHits_endY',
    'reco_daughter_PFP_true_byHits_endZ',
    'reco_daughter_PFP_true_byHits_pX',
    'reco_daughter_PFP_true_byHits_pY',
    'reco_daughter_PFP_true_byHits_pZ',
    'reco_daughter_PFP_true_byHits_startE',
    'reco_daughter_PFP_true_byHits_mass',
    'reco_daughter_PFP_true_byHits_sharedHits',
    'reco_daughter_PFP_true_byHits_sharedHits_collection',
    'reco_daughter_PFP_true_byHits_nHits',
    'reco_daughter_PFP_true_byHits_nHits_collection',
    'reco_daughter_PFP_true_byHits_hitsInRecoCluster',
    'reco_daughter_PFP_true_byHits_hitsInRecoCluster_collection',
    'reco_daughter_PFP_true_byHits_EnergyByHits',
    'reco_daughter_PFP_true_byHits_hit_channel',
    'reco_daughter_PFP_true_byHits_hit_peakTime',
    'reco_daughter_PFP_true_byHits_hit_integral',
    'reco_daughter_PFP_true_byHits_hit_energy',
    'reco_daughter_PFP_true_byHits_spacePointX',
    'reco_daughter_PFP_true_byHits_spacePointY',
    'reco_daughter_PFP_true_byHits_spacePointZ',
    'g4_num',
    'g4_mother',
    'g4_Pdg',
    'g4_startE',
    'g4_mass',
    'g4_startX',
    'g4_startY',
    'g4_startZ',
    'g4_endX',
    'g4_endY',
    'g4_endZ',
    'g4_pX',
    'g4_pY',
    'g4_pZ',
    'Run',
    'SubRun',
    'EventID',
    'totalEvents',
    'beamEvents'
]

/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:55: UserWarning: Couldn't apply filters to filters.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:55: UserWarning: Couldn't apply filters to filename.
  warnings.warn(f"Couldn't apply filters to {var}.")
/home/sb16165/Documents/pi0-analysis/analysis/python/analysis/Master.py:55: UserWarning: Couldn't apply filters to _TrueParticleData__pi0_MC.
  warnings.warn(f"Couldn't apply filters to {var}.")


'BeamMCFilter' executed in 0.5469s

number of dalitz decays: 0

'ApplyBeamFilter' executed in 0.5057s

...

False

True

True

True

'SplitSample' executed in 0.6507s

In [2]:
start_shower_indices = ak.local_index(events.recoParticles.number)[np.logical_or(*start_showers)]
print(start_shower_indices)

for i in range(ak.num(start_shower_indices, 0)):
    e1, e2 = GetSharedEnergy(events.trueParticlesBT, i, start_shower_indices[i][0], start_shower_indices[i][1])
    if e1 <= 0 or e2 <= 0: continue
    print("------------------------------------------------------------------------------------------------------")
    print(i)
    print(start_shower_indices[i])
    mask = GetSharedHitsMask(events.trueParticlesBT, i, start_shower_indices[i][0], start_shower_indices[i][1])
    print(f"count: {ak.count(mask[mask])}")
    print(f"shared energies: {e1}, {e2}")
    print(f"difference in shared energy calculation: {e1 - e2}")

    print(f"true pdg: {events.trueParticlesBT.pdg[i][start_shower_indices[i][0]]}, {events.trueParticlesBT.pdg[i][start_shower_indices[i][1]]}")
    true_energy = [events.trueParticlesBT.energy[i][start_shower_indices[i][j]] for j in range(2)]
    print(f"true energies: {true_energy}")
    true_energy_hits = [events.trueParticlesBT.energyByHits[i][start_shower_indices[i][j]] for j in range(2)]
    print(f"true energies by hits: {true_energy_hits}")

    residuals = [true_energy_hits[j] - true_energy[j] for j in range(2)]
    print(f"resiudals: {residuals}")
    print(f"total residual: {residuals[0] + residuals[1]}")
    print(f"total residual after removing shared hit energy: {residuals[0] + residuals[1] - e1}")
    print(f"total residual after removing shared hit energy: {residuals[0] + residuals[1] - e2}")


NameError: name 'events' is not defined

In [4]:
start_shower_indices = ak.local_index(events.recoParticles.number)[np.logical_or(*start_showers)]
print(start_shower_indices)

eve = 1

print(start_shower_indices[eve])

mask_1 = GetSharedHitsMask(events.trueParticlesBT, eve, start_shower_indices[eve][0], start_shower_indices[eve][1])
mask_0 = GetSharedHitsMask(events.trueParticlesBT, eve, start_shower_indices[eve][1], start_shower_indices[eve][0])

c_1 = events.trueParticlesBT.channel[eve][start_shower_indices[eve][1]][mask_1]
c_0 = events.trueParticlesBT.channel[eve][start_shower_indices[eve][0]][mask_0]

t_1 = events.trueParticlesBT.peakTime[eve][start_shower_indices[eve][1]][mask_1]
t_0 = events.trueParticlesBT.peakTime[eve][start_shower_indices[eve][0]][mask_0]

e_1 = events.trueParticlesBT.hit_energy[eve][start_shower_indices[eve][1]][mask_1]
e_0 = events.trueParticlesBT.hit_energy[eve][start_shower_indices[eve][0]][mask_0]

sps_1 = events.trueParticlesBT.spacePoint[eve][start_shower_indices[eve][1]][mask_1]
sps_0 = events.trueParticlesBT.spacePoint[eve][start_shower_indices[eve][0]][mask_0]

for h_1, e1, s_1 in zip(zip(c_1, t_1), e_1, sps_1):
    for h_0, e0, s_0 in zip(zip(c_0, t_0), e_0, sps_0):
        if h_0 == h_1 and e0 != e1:
            print(f"{h_0}, {e0}, {s_0}")
            print(f"{h_1}, {e1}, {s_1}")


[[32, 100], [35, 158]]

[35, 158]

(2247, 4451.28857421875), 0.3159021115600794, {x: -999, y: -999, z: -999}

(2247, 4451.28857421875), 0.31535729969001414, {x: -999, y: -999, z: -999}

(2247, 4440.06787109375), 1.7821858282006096, {x: -999, y: -999, z: -999}

(2247, 4440.06787109375), 1.7791122305311584, {x: -999, y: -999, z: -999}

(2209, 4509.42236328125), 1.4070887296341879, {x: -999, y: -999, z: -999}

(2209, 4509.42236328125), 1.4026474766856099, {x: -999, y: -999, z: -999}

In [42]:
def GetSharedHitsInSliceMask(events, e, a, slices):
    pfos_in_slice = np.logical_or(*[events.recoParticles.sliceID == slices[i] for i in range(len(slices))])
    pfo_slice_indices = ak.local_index(events.recoParticles.number)[pfos_in_slice][e]

    mask = None
    for i in pfo_slice_indices:
        if i == a: continue
        if events.trueParticlesBT.number[eve][i] == events.trueParticlesBT.number[eve][a]: continue
        if mask is None:
            mask = GetSharedHitsMask(events.trueParticlesBT, e, i, a)
        else:
            mask | GetSharedHitsMask(events.trueParticlesBT, e, i, a)
    return mask

eve = 0
pfo_indices = ak.local_index(events.recoParticles.number)

start_shower_index = pfo_indices[np.logical_or(*start_showers)][eve]
print(start_shower_index)

start_shower_slices = events.recoParticles.sliceID[np.logical_or(*start_showers)][eve]
print(start_shower_slices)

mask_0 = GetSharedHitsMask(events.trueParticlesBT, eve, start_shower_index[1], start_shower_index[0]) | GetSharedHitsInSliceMask(events, eve, start_shower_index[0], start_shower_slices)
mask_1 = GetSharedHitsMask(events.trueParticlesBT, eve, start_shower_index[0], start_shower_index[1]) | GetSharedHitsInSliceMask(events, eve, start_shower_index[1], start_shower_slices)

print(ak.count(mask_0[mask_0]))
print(ak.count(mask_1[mask_1]))
#? how to decide how much energy to take from the true Particle?


[32, 100]

[5, 5]

7

0

checking shared hits by space points was problematic, issues with additional ghost points being produced (I think) -> get link to talk about this

checking shared hits by space points works for all quantities check except the energy of the hit

checking shared hits by directly comparing the hit energies is the most consistent, but the values differ from either total energies found using the previous method

hit energy arrays not in the same order as other hit properties???

In [11]:
def StartShowerIndex(events : Master.Data, start_showers : list):
    return ak.local_index(events.recoParticles.number)[np.logical_or(*start_showers)][0]

event1 = Master.Data("pi0Test_output_mod_3.root")
event2 = Master.Data("pi0Test_output_mod.root")

LoadAdditionalNTuplesNew(event1)
LoadAdditionalNTuples(event2)

print(event1.trueParticlesBT.hitEnergy)
print(event2.trueParticlesBT.hitEnergy)

print(ak.count(event1.trueParticlesBT.hitEnergy))
print(ak.count(event2.trueParticlesBT.hitEnergy))

ss1, _ = merge_study.SplitSample(event1)
ss2, _ = merge_study.SplitSample(event2)

print(ak.all(ss1[0] == ss2[0]))
print(ak.all(ss1[1] == ss2[1]))

ssi1 = StartShowerIndex(event1, ss1)
ssi2 = StartShowerIndex(event2, ss2)

print(ssi1)
print(ssi2)

print(ak.count(event1.trueParticlesBT.hitEnergy[0][ssi1[0]]))
print(ak.count(event2.trueParticlesBT.hitEnergy[0][ssi2[0]]))

e_0 = ak.concatenate([ak.unflatten(event1.trueParticlesBT.hitEnergy[0][ssi1[0]], 1, -1), ak.unflatten(event2.trueParticlesBT.hitEnergy[0][ssi2[0]], 1, -1)], -1)
e_1 = ak.concatenate([ak.unflatten(event1.trueParticlesBT.hitEnergy[0][ssi1[1]], 1, -1), ak.unflatten(event2.trueParticlesBT.hitEnergy[0][ssi2[1]], 1, -1)], -1)

print(ak.to_list(e_0))
print(ak.to_list(e_1))


[... -999, -999, -999, -999, -999, -999, -999, -999, -999, -999, -999, -999]]]

[... -999, -999, -999, -999, -999, -999, -999, -999, -999, -999, -999, -999]]]

806832

806832

False

...

[35, 158]

653

653

[
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [0.4291906415990397, 0.4291906415990397],
    [1.7037708087196661, 1.7037708087196661],
    [0.4251540468456195, 0.4251540468456195],
    [2.438128170399865, 2.438128170399865],
    [3.2642053161476676, 3.2642053161476676],
    [2.554165031695729, 2.554165031695729],
    [0.8343603435266576, 0.8343603435266576],
    [5.157045508367986, 5.157045508367986],
    [0.32546755818181644, 0.32546755818181644],
    [2.0094860987679253, 2.0094860987679253],
    [0.3599583338437031, 0.3599583338437031],
    [1.6318321122472588, 1.6318321122472588],
    [1.6106193927664665, 1.6106193927664665],
    [0.23459126725535048, 0.23459126725535048],
    [1.5390433307663578, 1.5390433307663578],
    [0.5635749169246611, 0.5635749169246611],
    [1.520606083413921, 1.520606083413921],
    [0.24873415483136713, 0.24873415483136713],
    [1.4148421420034132, 1.4148421420034132],
    [0.613439670292379, 0.613439670292379],
    [1.6604781688252452, 1.6604781688252452],
    [1.6867628226688363, 1.6867628226688363],
    [1.5812671749542078, 1.5812671749542078],
    [0.17110497759735002, 0.17110497759735002],
    [0.32158902353631447, 0.32158902353631447],
    [0.48951714735578866, 0.48951714735578866],
    [0.4960147378873447, 0.4960147378873447],
    

[
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [-999.0, -999.0],
    [3.2847556796189883, 3.2847556796189883],
    [0.31535729969001414, 0.31535729969001414],
    [1.7791122305311584, 1.7791122305311584],
    [1.5011585555429978, 1.5011585555429978],
    [1.3040327941989487, 1.3040327941989487],
    [1.7600766508711316, 1.7600766508711316],
    [1.4026474766856099, 1.4026474766856099],
    [1.311114932654902, 1.311114932654902],
    [1.4973488591608055, 1.4973488591608055],
    [2.77840186023607, 2.77840186023607],
    [0.7413841117635963, 0.7413841117635963],
    [0.66568976791399, 0.66568976791399],
    [0.21790097710962777, 0.21790097710962777],
    [1.6753246468491119, 1.6753246468491119],
    [0.17018121968760716, 0.17018121968760716],
    [0.46978701558168806, 0.46978701558168806],
    [0.3172682622875175, 0.3172682622875175],
    [0.4891013187980854, 0.4891013187980854],
    [0.2962339801134727, 0.2962339801134727],
    [0.19040812749071878, 0.19040812749071878],
    [2.4327228530524025, 2.4327228530524025],
    [2.2020173580948703, 2.2020173580948703],
    [0.2537075643696791, 0.2537075643696791],
    [1.5955536187759074, 1.5955536187759074],
    [5.049991792221799, 5.049991792221799],
    [1.7301627564239292, 1.7301627564239292],
    [1.2699118786753691, 1.2699118786753691],
    [1.2442053817374366, 1.2442053817374366],
    [2.0633318373663303, 2.0633318373663303],
    [1.5181805657759466, 1.5181805657759466],
    [1.213508990119559, 1.213508990119559],
    [3.237024549449946, 3.237024549449946],
    [2.988427179318152, 2.988427179318152],
    [3.1157537936021678, 3.1157537936021678],
    [3.340225190679546, 3.340225190679546],
    [0.658579149575701, 0.658579149575701],
    [1.5719613489560957, 1.5719613489560957],
    [1.5025419710623227, 1.5025419710623227],
    [2.20465604771747, 2.20465604771747],
    [2.433361421840674, 2.433361421840674],
    [0.3681224629875016, 0.3681224629875016],
    [1.8191966311753545, 1.8191966311753545],
    [0.4010267175906412, 0.4010267175906412],
    [5.0349714931334955, 5.0349714931334955],
    [1.1305227009616157, 1.1305227009616157],
    [1.5663514484667294, 1.5663514484667294],
    [10.014909472620108, 10.014909472620108],
    [2.590034464773168, 2.590034464773168],
    [4.6655100641684, 4.6655100641684],
    [4.365086137984644, 4.365086137984644],
    [0.24578888374240726, 0.24578888374240726],
    [3.1475490193141193, 3.1475490193141193],
    [3.065079743697331, 3.065079743697331],
    [3.337025572740694, 3.337025572740694],
    